In [5]:
sys.path.append('../')
from __init__ import *

image = imread("../line_detector.tif")
image.shape
%matplotlib widget
%matplotlib inline


In [6]:
# cv2.namedWindow("src",cv2.WINDOW_AUTOSIZE)
# cv2.namedWindow("dst",cv2.WINDOW_AUTOSIZE)
%matplotlib widget
#%matplotlib inline
spatialRad=5
colorRad=1
maxPryLevel=1
src=image.copy()
dst=src.copy()
maxv=np.max(src)
img_norm=src.copy()
src=cv2.normalize(src, dst=img_norm,alpha=0,beta=255,norm_type=cv2.NORM_MINMAX)
src=src.astype('uint8')
src=cv2.cvtColor(src,cv2.COLOR_GRAY2BGR)
plt.imshow(src)

C:\Users\www_0\AppData\Roaming\Python\Python38\site-packages\ipywidgets\widgets\widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Canvas).__init__().
__init__() missing 1 required positional argument: 'figure'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)
C:\Users\www_0\AppData\Roaming\Python\Python38\site-packages\ipywidgets\widgets\widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Toolbar).__init__().
__init__() missing 1 required positional argument: 'canvas'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:

color_index=random_color_array(maxv)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 5, -0.5)
def on_meanshift(spatialRad,colorRad,maxLevel,criteria):
    dst=cv2.pyrMeanShiftFiltering(src,spatialRad,colorRad,maxLevel=maxLevel,termcrit=criteria)
    dst=cv2.cvtColor(dst,cv2.COLOR_BGR2GRAY)
    #label_dst=color_index[dst]
    return dst
#     cv2.imshow("dst",label_dst)

# width,height=image.shape
# cv2.createTrackbar("SpatialRad","dst",spatialRad,50,on_meanshift)
# cv2.createTrackbar("ColorRad","dst",colorRad,60,on_meanshift)
# cv2.createTrackbar("MaxPryLevel","dst",maxPryLevel,5,on_meanshift)
# cv2.imshow("dst",dst)
# cv2.waitKey()
_,ax=plt.subplots(4,4,sharex=True,sharey=True)
for i in range(0,4):
    for j in range(0,4):
        label_dst=on_meanshift(i*3,1,j,criteria)
        ax[i,j].imshow(label_dst,cmap="jet")
        text="s=%d,c=%d,p=%d"%(i*3,1,j)
        ax[i,j].set_title(text)
        ax[i,j].set_axis_off()
        
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:

_,ax=plt.subplots(4,4,sharex=True,sharey=True)
for i in range(0,4):
    for j in range(0,4):
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 1)
        label_dst=on_meanshift(i*2,j*2,1,criteria)
        ax[i,j].imshow(label_dst,cmap="jet")
        text="s=%d,c=%d,p=%d"%(i*2,j*2,1)
        ax[i,j].set_title(text)
        ax[i,j].set_axis_off()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
_,ax=plt.subplots(4,4,sharex=True,sharey=True)
for i in range(0,4):
    for j in range(0,4):
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, i*5,j*2-4)
        label_dst=on_meanshift(4,0,1,criteria)
        ax[i,j].imshow(label_dst,cmap="jet")
        text="iter=%d,accur=%d"%(i*3,j*2)
        ax[i,j].set_title(text)
        ax[i,j].set_axis_off()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# conclusion for opencv meanshift segment
1. py will influence(blur) segment ,need set 0-1 (recommend 0)
2. c will influence second , set 0-2
3. s is next, set 0-4
4. iter and precision is last, default is ok, can set 10,1 

# LK optical flow tracing


In [10]:
#(inverse compositional)
import math
images=imread("../cell0317-time.tif")

LK_images=[]

# conner detector params
feature_params=dict(maxCorners=100,qualityLevel=0.1,minDistance=7,blockSize=7)
# LK lightflow params
lk_params=dict(winSize=(15,15),maxLevel=2,criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.02))
frames,height,width=images.shape
tracks=[] # strore tran_len frames ' point moving path
track_len=15
frame_idx=0
detect_interval=1
while(frame_idx<frames):
    frame=images[frame_idx]
    src=cv2.normalize(frame,dst=img_norm,alpha=0,beta=255,norm_type=cv2.NORM_MINMAX)
    src=src.astype('uint8')
    frame=src.copy()
    vis=cv2.cvtColor(img_norm,cv2.COLOR_GRAY2BGR)
    if(len(tracks)>0):
        img0,img1=pregray,frame
        p0=np.float32([tr[-1] for tr in tracks]).reshape(-1,1,2)
        p1,st,err=cv2.calcOpticalFlowPyrLK(img0,img1,p0,None,**lk_params)
        p0r,st,err=cv2.calcOpticalFlowPyrLK(img1,img0,p1,None,**lk_params)
        d=abs(p0-p0r).reshape(-1,2).max(-1)
        #distance <1 is good track
        good=d>-1
        new_tracks=[]
        for i,(tr,(x,y),flag) in enumerate(zip(tracks, p1.reshape(-1, 2), good)):
            if not flag:
                continue
            tr.append((x,y))
            if len(tr)>track_len:
                del tr[0]
            new_tracks.append(tr) 
            if len(tr)<4:
                cv2.circle(vis,(int(x),int(y)),2,(255,0,0),-1)
            else:
                cv2.circle(vis,(int(x),int(y)),2,(0,255,0),-1)
    
        tracks=new_tracks
        cv2.polylines(vis,[np.int32(tr) for tr in tracks],1,(255,255,0),1)
    if frame_idx==0:
        mask=np.zeros_like(frame)
        mask[:]=255
        
        if frame_idx!=0:
            for x,y in [np.int32(tr[-1]) for tr in tracks ]:
                cv2.circle(mask, (int(x), int(y)), 5, 0, -1)
        p = cv2.goodFeaturesToTrack(frame, mask=mask, **feature_params)
        if p is not None:
            for x, y in np.float32(p).reshape(-1,2):
                tracks.append([(x, y)])
    frame_idx+=1
    pregray=frame
    LK_images.append(vis)

    
        
      

In [ ]:
# #from random_color import random_color_f,random_color_list
# # to open a tiff file for reading:

# print(images.shape)
# def random_color_list(n):
#     return [random_color.random_color_f() for i in range(n)]
# colors=random_color_list(13)
# def browse_images(images):
#     n=len(images)
   
#     def view_image(i):
#         fig,ax=plt.subplots()
#         ax.imshow(LK_images[i])
        
#         # plt.title('Training: %s' % i)
#         # plt.show()
#     interact(view_image, i=(0,n-1))
# browse_images(images)


In [11]:
LK_images[0].dtype
LK_images[0].shape

(512, 512, 3)

In [ ]:
from tifffile import imread,imwrite
import tifffile
imwrite( "Lkoflow.tif", np.array(LK_images) ,photometric='rgb',metadata={'axes': 'TYX'})

In [2]:
from __init__ import *
import cv2
import LucasKanade3D
#(inverse compositional)
import math
images=imread("cell0317-time.tif")

LK_images=[]

# conner detector params
feature_params=dict(maxCorners=100,qualityLevel=0.1,minDistance=7,blockSize=7)
# LK lightflow params
lk_params=dict(winSize=(15,15),maxLevel=2,criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.02))
frames,height,width=images.shape
tracks=[] # strore tran_len frames ' point moving path
track_len=15
frame_idx=0
detect_interval=1
while(frame_idx<frames):
    frame=images[frame_idx]
    img_norm=frame.copy()
    src=cv2.normalize(frame,dst=img_norm,alpha=0,beta=255,norm_type=cv2.NORM_MINMAX)
    src=src.astype('uint8')
    frame=src.copy()
    vis=cv2.cvtColor(img_norm,cv2.COLOR_GRAY2BGR)
    if(len(tracks)>0):
        img0,img1=pregray,frame
        p0=np.float32([tr[-1] for tr in tracks]).reshape(-1,1,2)
        #p1,st,err=LucasKanade3D.calcOpticalFlowPyrLK3D(img0, img1, p0, size=[11,11])
        p1,st,err=cv2.calcOpticalFlowPyrLK(img0,img1,p0,None,**lk_params)
        p0r,st,err=cv2.calcOpticalFlowPyrLK(img1,img0,p1,None,**lk_params)
        d=abs(p0-p0r).reshape(-1,2).max(-1)
        #distance <1 is good track
        good=d<3
        new_tracks=[]
        for i,(tr,(x,y),flag) in enumerate(zip(tracks, p1.reshape(-1, 2), good)):
            if not flag:
                continue
            tr.append((x,y))
            if len(tr)>track_len:
                del tr[0]
            new_tracks.append(tr) 
            if len(tr)<4:
                cv2.circle(vis,(int(x),int(y)),2,(255,0,0),-1)
            else:
                cv2.circle(vis,(int(x),int(y)),2,(0,255,0),-1)
    
        tracks=new_tracks
        cv2.polylines(vis,[np.int32(tr) for tr in tracks],1,(255,255,0),1)
    if frame_idx%detect_interval==0:
        mask=np.zeros_like(frame)
        mask[:]=255
        
        if frame_idx!=0:
            for x,y in [np.int32(tr[-1]) for tr in tracks ]:
                cv2.circle(mask, (int(x), int(y)), 5, 0, -1)
        p = cv2.goodFeaturesToTrack(frame, mask=mask, **feature_params)
        if p is not None:
            for x, y in np.float32(p).reshape(-1,2):
                tracks.append([(x, y)])
    frame_idx+=1
    pregray=frame
    LK_images.append(vis)



ModuleNotFoundError: No module named '__init__'